# 1.1 Struktura zbioru danych

In [ ]:
import pandas as pd
#Załaduj plik iris.data i przypisz go do Data Frame następnie go wyświetl
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(url, header=None)
print("----------------------------------------------------------------------------------------------------------------")
print("df is : {}".format(df)) #Wyświetl pojedynczy wzorzec.
print("----------------------------------------------------------------------------------------------------------------")
print("df.loc[0] is : {}".format(df.loc[0]))
print("----------------------------------------------------------------------------------------------------------------")

In [ ]:
data = df.values

#przestrzeń cech — zbiór wszystkich wektorów cech obiektów znajdujących się w zbiorze
X = data[:,:-1]
print("X is : {}".format(X))
print("----------------------------------------------------------------------------------------------------------------")
print("X.shape is : {}".format(X.shape))
print("----------------------------------------------------------------------------------------------------------------")

In [ ]:
#zbiór etykiet — zawierający etykiety dla obiektów znajdujących się w zbiorze 
y = data[:,-1]
print("y is : {}".format(y))
print("----------------------------------------------------------------------------------------------------------------")
print("y shape is : {}".format(y.shape))
print("----------------------------------------------------------------------------------------------------------------")

# 1.2 Podział zbioru danych na uczący i testowy

In [ ]:
# przydatne nam metody podziału
from sklearn import model_selection

#losowy podział zbioru na dwie części z zachowaniem  proporcji pomiędzy nimi.
#test_size  - proporcję pomiędzy podziałami.
#random_state - zapewnienie powtarzalności podziału 
splits = model_selection.train_test_split(X, y, test_size=.333, random_state=0)
X_train, X_test, y_train, y_test = splits
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#W podejściu krzyżowym dzielimy losowo zbiór danych na k możliwie równych sobie podzbiorów. 
#Uzyskujemy dzięki temu k par, w których każdy z podzbiorów raz występuje jako zbiór uczący
#a pozostała, połączona część zbioru jest wykorzystywana jako zbiór testowy.
k = 10
cv = model_selection.StratifiedKFold(n_splits=k)

fold = 0
for train, test in cv.split(X_train, y_train):
    print ("Fold %i (%i w TS, %i w VS)" % (fold, len(train), len(test)))
    print (train)
    fold += 1

# 1.3 Proces uczenia i predykcja
### W problemie klasyfikacji, naszym zadaniem jest stworzenie algorytmu (programu), który na podstawie znanych sobie, opisanych wzorców, będzie w stanie efektywnie rozpoznawać wzorce nieopisane i dotąd sobie nieznane.

In [ ]:
#algorytm k najbliższych sąsiadów
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier() #konkretny klasyfikator
clf.fit(X_train, y_train) #Wytrenuj klasyfikator korzystając z zbioru danych
score = clf.score(X_test, y_test) #Określ jakość klasyfikatora
print(score)

In [ ]:
#Wylicz wekotra wsparcia(Określa on prawdopodobieństwo przynależności obiektu do każdej z możliwych klas i zawsze sumuje się do jedynki)
support_vector = clf.predict_proba(X_test)
print(support_vector)

In [ ]:
import numpy as np

#Wynik klasyfikacji (przypisanie wzorca do klasy) nazywamy predykcją. Możemy ją uzyskać wyliczając indeks najwyższej wartości w wektorze wsparcia 
prediction = np.argmax(support_vector,axis = 1)
print(prediction)

In [ ]:
#Posłuż się atrybutem classes_ "przetłumaczenia" indeksów klas predykcji na odpowiadające im łańcuchy znakowe
prediction = clf.classes_[prediction]
print(prediction)

# 1.4 Miary oceny jakości klasyfikacji

In [15]:
#Narzędzia do oceny jakości
from sklearn import metrics

# Ocena jakości klasyfikacji polega na odpowiednim porównaniu ze sobą tych dwóch wektorów
# Podstatowym narzędziem takiej oceny jest tak zwana macierz pomyłek
confusion_matrix = metrics.confusion_matrix(y_test, prediction)
print(confusion_matrix)

[[16  0  0]
 [ 0 18  1]
 [ 0  0 15]]


In [16]:
#dokładność, czyli stosunek sumy wartości znajdujących się na przekątnej macierzy pomyłek do sumy wszystkich wartości macierzy. 
accuracy = metrics.accuracy_score(y_test, prediction)
print (accuracy)

0.98


In [18]:
k = 10 # 10 podziałów. 
cv = model_selection.StratifiedKFold(n_splits=k) #obiekt podziału

accuracies = [] #kolejne wyliczone dokładności
for train, test in cv.split(X, y):
    #Odpowiednie podzbiory tablic
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]

    clf.fit(X_train, y_train) #ucz klasyfikator
    probas = clf.predict_proba(X_test) #wyliczamy wektory wsparć
    # wyliczamy i tłumaczymy predykcję i wyliczamy dokładność
    prediction = np.argmax(probas,axis = 1)
    prediction = clf.classes_[prediction]
    accuracy = metrics.accuracy_score(y_test, prediction)
    accuracies.append(accuracy)

print (accuracies)

[1.0, 0.9333333333333333, 1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 0.9333333333333333, 1.0, 1.0, 1.0]


In [19]:
#Uśredniona dokładność i odchylenie standardowe.
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
print ("Średnia dokładność %.3f (+- %.2f)" % (mean_accuracy, std_accuracy))

Średnia dokładność 0.967 (+- 0.04)


# 1.5 Porównywanie klasyfikatorów

In [20]:
from sklearn import neighbors
#lista clfs zawierające dwa różne klasyfikatory k-NN
clfs = [
    neighbors.KNeighborsClassifier(n_neighbors = 1),
    neighbors.KNeighborsClassifier(n_neighbors = 5)
]

#wczytamy zbiór danych, dzieląc go na przestrzeń cech i zbiór etykiet
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(url, header=None)
data = df.values
X = data[:,:-1]
y = data[:,-1]

#Podział zbioru z użyciem k-foldowej walidacji krzyżowej
k = 10
cv = model_selection.StratifiedKFold(n_splits=k)

results = []
for train, test in cv.split(X,y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    result = []
    for clf in clfs:
        clf.fit(X_train, y_train)
        probas = clf.predict_proba(X_test)
        prediction = np.argmax(probas,axis = 1)
        prediction = clf.classes_[prediction]
        accuracy = metrics.accuracy_score(y_test, prediction),
        result.append(accuracy[0])
    results.append(result)
results = np.array(results)

print(results)

[[1.         1.        ]
 [0.93333333 0.93333333]
 [1.         1.        ]
 [0.93333333 1.        ]
 [0.86666667 0.86666667]
 [1.         0.93333333]
 [0.86666667 0.93333333]
 [1.         1.        ]
 [1.         1.        ]
 [1.         1.        ]]


In [22]:
#testy statystyczne
from scipy import stats

#Wykonaj testy przekazując  kolumny tablicy wyników odpowiadające porównywanym klasyfikatorom
test_t = stats.ttest_ind(results[:,0],results[:,1])
test_w = stats.wilcoxon(results[:,0],results[:,1])
print (np.mean(results,axis=0))

[0.96       0.96666667]


C:\Users\Ragnus\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2778: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")


In [23]:
#sprawdź p-wartości uzyskanych testów
print (test_t)
print (test_w)

Ttest_indResult(statistic=-0.2873478855663521, pvalue=0.7771278487505224)
WilcoxonResult(statistic=2.0, pvalue=0.5637028616507731)
